## House Price Prediction With MLflow

In this tutorial we will:

    - Run hyperparameter tuning while training a model
    - Log every hyperparameter and metrics in the MLflow UI
    - Compares the results of the various runs in the MLflow UI
    - Choose the best run and regester it as a best model

In [3]:
import pandas as pd
import numpy as np

import mlflow
import mlflow.sklearn

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.datasets import fetch_california_housing


In [5]:
housing = fetch_california_housing()
housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]], shape=(20640, 8)),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894], shape=(20640,)),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': 

In [8]:
## Preparing the dataset

data = pd.DataFrame(housing.data, columns=housing.feature_names)
data["Price"] = housing.target
data

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Price
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


In [11]:
# Checking for any null values.

data_null = data.isnull().sum()
data_null

MedInc        0
HouseAge      0
AveRooms      0
AveBedrms     0
Population    0
AveOccup      0
Latitude      0
Longitude     0
Price         0
dtype: int64

No Null Values are present in the dataset.

# Doing Train Test Split, Model Hyperparameter tuning, MLflow experiments

In [18]:
from urllib.parse import urlparse
from mlflow.models import infer_signature

In [ ]:
## Doing Train Test Split, Model Hyperparameter tuning, MLflow experiments

X = data.drop(columns = "Price", axis=1)
y = data["Price"]


In [27]:
# Hyperparameter Tuning using GridSearch CV

def hyperparameter_tuning(X_train,y_train, hyperparameter_grid):
    rf = RandomForestRegressor()
    grid_search = GridSearchCV(estimator=rf, param_grid = hyperparameter_grid, cv=3, n_jobs=-1, verbose=2,
                                scoring = "neg_mean_squared_error")
                            
    grid_search.fit(X_train,y_train)
    return grid_search

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

signature = infer_signature(X_train, y_train)


hyperparameters_grid = {
    "n_estimators": [200, 300, 500],
    "max_depth": [6, 9, 12, 16],
    "min_samples_split": [5, 7, 12],
    "min_samples_leaf": [6, 7, 9]
}

## Start the MLflow Experiment

with mlflow.start_run():
    ## Perform Hyperparameter Tuning
    grid_search = hyperparameter_tuning(X_train,y_train, hyperparameters_grid)

    ## Get the best model

    best_model = grid_search.best_estimator_

    ## Evaluating the best model

    y_pred = best_model.predict(X_test)
    mae = mean_absolute_error(y_test,y_pred)


    ## Log the best parameters and metrics
    mlflow.log_param("best_n_estimators", grid_search.best_params_['n_estimators'])
    mlflow.log_param("best_max_depth", grid_search.best_params_['max_depth'])
    mlflow.log_param("best_min_samples_split", grid_search.best_params_['min_samples_split'])
    mlflow.log_param("best_min_samples_leaf", grid_search.best_params_['min_samples_leaf'])
    mlflow.log_metric("mae", mae)


    ## Tracking URL
    mlflow.set_tracking_uri("http://127.0.0.1:5000")
    tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme

    if tracking_url_type_store !='file':
        mlflow.sklearn.log_model(best_model,"model", registered_model_name = "Best Randomforest Model")
    else:
        mlflow.sklearn.log_model(best_model,"model", signature=signature)

    print(f"Best Parameters: {grid_search.best_params_}")
    print(f"Mean Absolute Error: {mae}")



Fitting 3 folds for each of 108 candidates, totalling 324 fits


MlflowException: API request to http://127.0.0.1:5000/api/2.0/mlflow/runs/get failed with exception HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/2.0/mlflow/runs/get?run_uuid=c3e04aa20dd64e499b22e627af0086b6&run_id=c3e04aa20dd64e499b22e627af0086b6 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000174CC1B7ED0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))